In [1]:
import os
import re
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import requests
import time

# ---------------------------------------------------
# CONFIGURATION DU DOSSIER DE CV
# ---------------------------------------------------
cv_folder = r"data\CVs"

if not os.path.exists(cv_folder):
    raise FileNotFoundError(f"Le dossier {cv_folder} est introuvable.")

# Mode d'appel : "ollama" (local) ou "huggingface"
LLM_MODE = "ollama"

# ---------------------------------------------------
# FONCTION : extraire le texte depuis un PDF
# ---------------------------------------------------
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text("text")
    except Exception as e:
        print(f"⚠️ Erreur lecture PDF {pdf_path}: {e}")
    return text


# ---------------------------------------------------
# FONCTIONS HEURISTIQUES CLASSIQUES
# ---------------------------------------------------
def extract_name(text):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return np.nan
    for line in lines[:15]:
        clean_line = re.sub(r"[^A-Za-zÀ-ÿ\s\-]", "", line)
        if len(clean_line.split()) == 2 and clean_line.isupper():
            return clean_line.title()
    for line in lines[:15]:
        match = re.match(
            r"([A-ZÉÈÊËÎÏÔÖÙÜÂÄÇ][a-zàâäéèêëîïôöùüûç\-']+\s+[A-ZÉÈÊËÎÏÔÖÙÜÂÄÇ][a-zàâäéèêëîïôöùüûç\-']+)",
            line
        )
        if match:
            return match.group(1).strip()
    for i in range(len(lines)-1):
        if lines[i].isupper() and lines[i+1].isupper():
            return f"{lines[i].title()} {lines[i+1].title()}"
    return np.nan


def extract_poste(text):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    for line in lines[:15]:
        if re.search(
            r"(chargé|accompagnateur|manager|chef|consultant|analyste|développeur|technicien|assistant|ingénieur|responsable)",
            line,
            re.IGNORECASE
        ):
            return line.strip()
    match = re.search(
        r"(Chargé|Accompagnateur|Manager|Chef|Consultant|Analyste|Technicien|Assistant|Ingénieur|Responsable)[^\n]{0,60}",
        text,
        re.IGNORECASE
    )
    return match.group(0).strip() if match else np.nan


def extract_email(text):
    match = re.search(r"[\w\.-]+@[\w\.-]+\.\w+", text)
    return match.group(0) if match else np.nan


def extract_phone(text):
    match = re.search(r"(\+?\d[\d\s\-\(\)]{8,})", text)
    return match.group(0) if match else np.nan


def extract_languages(text):
    langs = re.findall(r"(Anglais|Français|Allemand|Espagnol|Italien|Arabe|Portugais)", text, re.IGNORECASE)
    return ", ".join(sorted(set([l.capitalize() for l in langs]))) if langs else np.nan


def extract_competences(text):
    comp_lines = []
    if "compétence" in text.lower():
        lines = text.splitlines()
        start = [i for i, l in enumerate(lines) if "compétence" in l.lower()]
        for s in start:
            for l in lines[s+1:s+8]:
                if not l.strip() or ":" in l:
                    break
                comp_lines.append(l.strip("•- ").strip())
    elif re.search(r"Python|SQL|Excel|Power BI|Gestion|Communication|Leadership", text, re.IGNORECASE):
        matches = re.findall(
            r"(Python|SQL|Excel|Power BI|Machine Learning|Communication|Leadership|Gestion)",
            text,
            re.IGNORECASE
        )
        comp_lines = list(set(matches))
    return ", ".join(comp_lines) if comp_lines else np.nan


def extract_formation(text):
    matches = re.findall(r"(Master|Licence|Doctorat|BTS|DUT|MBA)[^\n]{0,60}", text, re.IGNORECASE)
    return " | ".join(matches) if matches else np.nan


def extract_experience(text):
    matches = re.findall(r"(Chargé|Assistant|Chef|Manager|Stagiaire|Consultant|Ingénieur)[^\n]{0,80}", text, re.IGNORECASE)
    return " | ".join(matches) if matches else np.nan


# ---------------------------------------------------
# FONCTION : appel du modèle LLaMA 3.2:3B
# ---------------------------------------------------
def llama3_extract_fields(text):
    # On limite le texte pour accélérer la génération
    text = text[:1500]

    prompt = f"""
Tu es un expert RH. Voici un extrait de CV :

{text}

Extrait uniquement un objet JSON avec ces champs :
{{
 "nom complet": "",
 "poste visé": "",
 "email": "",
 "téléphone": "",
 "langues parlées": "",
 "compétences clés": "",
 "formation": "",
 "expériences principales": ""
}}
    """

    if LLM_MODE == "ollama":
        try:
            response = requests.post(
                "http://localhost:11434/api/generate",
                json={"model": "llama3.2:3b", "prompt": prompt, "stream": False},
                timeout=900  # 15 minutes max
            )
            if response.status_code == 200:
                raw_text = response.json().get("response", "")
                json_match = re.search(r"\{.*\}", raw_text, re.DOTALL)
                if json_match:
                    return json.loads(json_match.group(0))
                else:
                    print("⚠️ Aucun JSON détecté dans la réponse LLaMA.")
        except requests.exceptions.Timeout:
            print("⚠️ Timeout : LLaMA 3 a mis trop de temps à répondre (900s).")
        except Exception as e:
            print(f"⚠️ Erreur Ollama: {e}")

    return {}


# ---------------------------------------------------
# TRAITEMENT DE TOUS LES CV
# ---------------------------------------------------
rows = []
print("🧠 Vérification du modèle LLaMA 3.2:3b...")
os.system("ollama ps")
time.sleep(2)

print("\n🚀 Démarrage de l’analyse des CVs...\n")
for i, file in enumerate(tqdm(os.listdir(cv_folder), desc="Analyse des CVs")):
    if file.lower().endswith(".pdf"):
        path = os.path.join(cv_folder, file)
        text = extract_text_from_pdf(path)

        nom_h = extract_name(text)
        poste_h = extract_poste(text)
        email_h = extract_email(text)
        tel_h = extract_phone(text)

        # Si les heuristiques ne trouvent rien → appel du LLM
        if pd.isna(nom_h) or pd.isna(poste_h):
            llm_data = llama3_extract_fields(text)
        else:
            llm_data = {}

        metadata = {
            "Nom": nom_h if pd.notna(nom_h) else llm_data.get("nom complet", np.nan),
            "Poste": poste_h if pd.notna(poste_h) else llm_data.get("poste visé", np.nan),
            "Email": email_h or llm_data.get("email", np.nan),
            "Téléphone": tel_h or llm_data.get("téléphone", np.nan),
            "Langues": extract_languages(text) or llm_data.get("langues parlées", np.nan),
            "Compétences": extract_competences(text) or llm_data.get("compétences clés", np.nan),
            "Formation": extract_formation(text) or llm_data.get("formation", np.nan),
            "Expériences": extract_experience(text) or llm_data.get("expériences principales", np.nan),
            "Lien": path,
        }

        rows.append(metadata)
        print(f"✅ CV traité ({i+1}) : {file}")

# ---------------------------------------------------
# CRÉATION DU DATAFRAME ET SAUVEGARDE
# ---------------------------------------------------
df = pd.DataFrame(rows)
df.replace("", np.nan, inplace=True)

output_path = os.path.join(cv_folder, "cv_metadata_llama3.xlsx")
df.to_excel(output_path, index=False, engine="openpyxl")

print(f"\n✅ Extraction terminée avec LLaMA 3.2:3B. Fichier sauvegardé : {output_path}")
print(df.head(10))


🧠 Vérification du modèle LLaMA 3.2:3b...

🚀 Démarrage de l’analyse des CVs...



Analyse des CVs:   0%|          | 0/30 [00:00<?, ?it/s]

✅ CV traité (1) : cv1.pdf
✅ CV traité (2) : cv10.pdf


Analyse des CVs:  23%|██▎       | 7/30 [05:04<13:22, 34.89s/it] 

✅ CV traité (3) : cv11.pdf
✅ CV traité (4) : cv12.pdf
✅ CV traité (5) : cv13.pdf
✅ CV traité (6) : cv14.pdf
✅ CV traité (7) : cv15.pdf
✅ CV traité (8) : cv16.pdf
✅ CV traité (9) : cv17.pdf
✅ CV traité (10) : cv18.pdf


...

Analyse des CVs:  97%|█████████▋| 29/30 [36:41<01:21, 81.80s/it]

✅ CV traité (29) : cv8.pdf


Analyse des CVs: 100%|██████████| 30/30 [41:43<00:00, 83.45s/it] 

✅ CV traité (30) : cv9.pdf



✅ Extraction terminée avec LLaMA 3.2:3B. Fichier sauvegardé : data\CVs\cv_metadata_llama3.xlsx
                                         Nom  \
0        Nicolas Thomas Guide Accompagnateur   
1                              Ismael Panier   
2                                Reda Aboudi   
3  Formation Scolaire : Langues Et Hobbies :   
4                                    Sql Php   
5                            Stage Assistant   
6                             Data Analytics   
7                                   B Toeic    
8                                Samar Chari   
9            C E N T R E S D ’ I N T É R Ê T   

                                               Poste  \
0                                     ACCOMPAGNATEUR   
1  Développeur Full Stack  (2 Semaine en entrepri...   
2  Assistant Développeur Web / Support Technique ...   
3   Septembre 2023 à août 2025 : Apprenti technicien   
4                            Technicien Supérieur en   
5  Stage Assistant Ing´enieur Informati